# Karta från geojson toilets_and_water_SAT_2025-07-24_0713

* [Stockholm_Archipelago_Trail/issues/117](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/117) "Generate objects along the Trail - drinking water / toilets"
* [Stockholm_Archipelago_Trail/issues/122](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/122) "Statisk genererad websida med problem"


---- 
#### Karta uppladdad till GITHUB
* [toaletter_vatten_sat_karta_latest](https://raw.githack.com/salgo60/Stockholm_Archipelago_Trail/main/notebook/output/toaletter_vatten_sat_karta_latest.html)
* [Stockholm_Archipelago_Trail/html/sat_karta_komplett.html](https://salgo60.github.io/Stockholm_Archipelago_Trail/html/sat_karta_komplett.html)


* [Stockholm_Archipelago_Trail/html/toaletter_vatten_sat_karta.html](https://salgo60.github.io/Stockholm_Archipelago_Trail/html/toaletter_vatten_sat_karta.html)


In [1]:
import time
from datetime import datetime

start_time = time.time()
print("Start:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))


Start: 2025-09-16 04:10:07


In [2]:
import folium
import requests
from folium import IFrame
from folium.plugins import Fullscreen
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql" 
# === 1. Hämta toaletter och vatten från Gist ===
geojson_url = "https://gist.githubusercontent.com/salgo60/e16b01890b2ac276342f473c23a7187a/raw/toilets_and_water_SAT_2025-07-24_0713.geojson"
geojson_data = requests.get(geojson_url).json()

# === 2. Skapa karta ===
m = folium.Map(location=[59.3, 18.5], zoom_start=8, control_scale=True)
Fullscreen().add_to(m)

# === 3. Lägg till toaletter & vatten ===
def create_popup(properties):
    name = str(properties.get("name") or "⚠️ Namn saknas")
    wikidata = properties.get("wikidata")
    osm_id = properties.get("osm_id")
    osm_type = properties.get("osm_type", "node")
    website = properties.get("website")
    image = properties.get("image")

    html = f"<b>{name}</b><br>"

    if image:
        html += f'<img src="{image}" width="200"><br>'

    if wikidata:
        html += f'🧠 <a href="https://www.wikidata.org/wiki/{wikidata}" target="_blank">{wikidata}</a><br>'
    else:
        html += "⚠️ Saknar Wikidata<br>"

    if website:
        html += f'🌐 <a href="{website}" target="_blank">{website}</a><br>'
    else:
        html += "⚠️ Saknar webbsida<br>"

    if osm_id:
        osm_url = f"https://www.openstreetmap.org/{osm_type}/{osm_id}"
        osm_edit = f"https://www.openstreetmap.org/edit?{osm_type}={osm_id}"
        html += f'🧭 <a href="{osm_url}" target="_blank">OSM</a><br>'
        html += f'✏️ <a href="{osm_edit}" target="_blank">Redigera i OSM</a><br>'
    else:
        html += "⚠️ Saknar OSM-id<br>"

    html += "<hr><b>📋 Förbättringsförslag:</b><ul>"
    if not image:
        html += "<li>⚠️ Lägg till bild</li>"
    if not website:
        html += "<li>⚠️ Lägg till webbsida</li>"
    if not wikidata:
        html += "<li>⚠️ Lägg till Wikidata</li>"
    html += "</ul>"

    etapp_namn = str(properties.get("name") or "")
    if etapp_namn.startswith("SAT_"):
        commons_url = f"https://wikimap.toolforge.org/?cat={etapp_namn}&subcats=true&subcatdepth=3&cluster=false"
        html += f'🖼️ <a href="{commons_url}" target="_blank">Commons bilder</a><br>'

    return folium.Popup(IFrame(html, width=280, height=320))

for feature in geojson_data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    lon, lat = coords if feature["geometry"]["type"] == "Point" else coords[0][0]

    amenity = props.get("amenity", "")
    if amenity == "toilets":
        icon = folium.Icon(color="blue", icon="restroom", prefix="fa")
    elif amenity == "drinking_water":
        icon = folium.Icon(color="green", icon="tint", prefix="fa")
    else:
        icon = folium.Icon(color="gray", icon="question", prefix="fa")

    folium.Marker(
        location=[lat, lon],
        popup=create_popup(props),
        icon=icon
    ).add_to(m)

# === 4. Hämta etapper från Wikidata ===
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setQuery("""
SELECT ?etapp ?etappLabel ?osmID WHERE {
  ?etapp wdt:P361 wd:Q131318799;
         wdt:P31 wd:Q2143825;
         wdt:P402 ?osmID.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "sv,en". }
}
ORDER BY ?etappLabel
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

etapper = []
for r in results["results"]["bindings"]:
    etapper.append({
        "name": r["etappLabel"]["value"],
        "wikidata": r["etapp"]["value"],
        "osm_id": r["osmID"]["value"]
    })

# === 5. Rita varje etapp från Overpass ===
overpass_url = "https://overpass-api.de/api/interpreter"
for e in etapper:
    osm_id = e["osm_id"]
    name = e["name"]
    qid = e["wikidata"].split("/")[-1]
    osm_url = f"https://www.openstreetmap.org/relation/{osm_id}"
    wikidata_url = f"https://www.wikidata.org/wiki/{qid}"
    commons_url = f"https://wikimap.toolforge.org/?cat=SAT_{name.replace(' ', '_')}&subcats=true&subcatdepth=3&cluster=false"

    query = f"""
    [out:json][timeout:30];
    relation({osm_id});
    out geom;
    """
    r = requests.get(overpass_url, params={'data': query})
    data = r.json()
    rels = [el for el in data['elements'] if el['type'] == 'relation' and 'geometry' in el]

    for rel in rels:
        coords = [(p['lat'], p['lon']) for p in rel['geometry']]
        popup_html = f"""
        <b>{name}</b><br>
        🧭 <a href="{osm_url}" target="_blank">OSM</a><br>
        🧠 <a href="{wikidata_url}" target="_blank">Wikidata</a><br>
        🖼️ <a href="{commons_url}" target="_blank">Commons-karta</a><br>
        """
        folium.PolyLine(coords, color="red", weight=4, tooltip=name,
                        popup=folium.Popup(popup_html, max_width=300)).add_to(m)

# === 6. Exportera till HTML ===
output_file = "sat_karta_komplett.html"
m.save(output_file)
print(f"Karta sparad: {output_file}")


Karta sparad: sat_karta_komplett.html


In [3]:
m

In [4]:
import folium
import requests
from folium import IFrame
from folium.plugins import Fullscreen
import os

# === 1. Toaletter och vatten från GeoJSON ===
data_url = "https://gist.githubusercontent.com/salgo60/e16b01890b2ac276342f473c23a7187a/raw/toilets_and_water_SAT_2025-07-24_0713.geojson"
geojson_data = requests.get(data_url).json()

# === 2. Skapa karta ===
m = folium.Map(location=[59.3, 18.5], zoom_start=9, control_scale=True)
Fullscreen().add_to(m)

# === 3. Popup-generator ===
def create_popup(properties):
    name = properties.get("name", "⚠️ Namn saknas")
    wikidata = properties.get("wikidata")
    osm_id = properties.get("osm_id")
    osm_type = properties.get("osm_type", "node")
    website = properties.get("website")
    image = properties.get("image")

    html = f"<b>{name}</b><br>"

    if image:
        html += f'<img src="{image}" width="200"><br>'

    if wikidata:
        html += f'Wikidata: <a href="https://www.wikidata.org/wiki/{wikidata}" target="_blank">{wikidata}</a><br>'
    else:
        html += "⚠️ Saknar Wikidata<br>"

    if website:
        html += f'Website: <a href="{website}" target="_blank">{website}</a><br>'
    else:
        html += "⚠️ Saknar webbsida<br>"

    if osm_id:
        osm_url = f"https://www.openstreetmap.org/{osm_type}/{osm_id}"
        osm_edit = f"https://www.openstreetmap.org/edit?{osm_type}={osm_id}"
        html += f'OSM: <a href="{osm_url}" target="_blank">{osm_url}</a><br>'
        html += f'✏️ <a href="{osm_edit}" target="_blank">Redigera i OSM</a><br>'
    else:
        html += "⚠️ Saknar OSM-id<br>"

    html += "<hr><b>📋 Förbättringsförslag:</b><ul>"
    if not image:
        html += "<li>⚠️ Lägg till bild</li>"
    if not website:
        html += "<li>⚠️ Lägg till officiell webbsida</li>"
    if not wikidata:
        html += "<li>⚠️ Lägg till Wikidata-koppling</li>"
    if not osm_id:
        html += "<li>⚠️ Lägg till OSM-länk</li>"
    html += "</ul>"
    etapp_namn = str(properties.get("name") or "")
    if etapp_namn.startswith("SAT_"):
        wiki_url = f"https://wikimap.toolforge.org/?cat={etapp_namn}&subcats=true&subcatdepth=3&cluster=false"  
        html += f'🖼️ <a href="{wiki_url}" target="_blank">Wikimedia Commons bilder</a><br>'
    #print("Etappnamn:", etapp_namn)
    return folium.Popup(IFrame(html, width=260, height=300))

# === 4. Lägg till markörer ===
for feature in geojson_data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    lon, lat = coords if feature["geometry"]["type"] == "Point" else coords[0][0]

    amenity = props.get("amenity", "")
    if amenity == "toilets":
        icon = folium.Icon(color="blue", icon="restroom", prefix="fa")
    elif amenity == "drinking_water":
        icon = folium.Icon(color="green", icon="tint", prefix="fa")
    else:
        icon = folium.Icon(color="gray", icon="question", prefix="fa")

    folium.Marker(
        location=[lat, lon],
        popup=create_popup(props),
        icon=icon
    ).add_to(m)

# === 5. Ladda och rita SAT-leden via Overpass ===
overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
relation(19012437);
way(r);
(._;>;);
out body;
"""
r = requests.get(overpass_url, params={'data': overpass_query})
data = r.json()

# Extrahera koordinater
nodes = {el['id']: (el['lat'], el['lon']) for el in data['elements'] if el['type'] == 'node'}
ways = [el for el in data['elements'] if el['type'] == 'way']

for way in ways:
    coords = [(nodes[nid][0], nodes[nid][1]) for nid in way['nodes'] if nid in nodes]
    folium.PolyLine(locations=coords, color="red", weight=4, opacity=0.8, tooltip="SAT-leden").add_to(m)

# === 6. Spara som HTML ===
output_path = "toaletter_vatten_sat_karta_2.html"
m.save(output_path)
print(f"Karta sparad som: {output_path}")


Karta sparad som: toaletter_vatten_sat_karta_2.html


In [5]:
# Visa kartan
m

### New version 2025-09-15 

In [6]:
import folium
import requests
from folium import IFrame
from folium.plugins import Fullscreen, FloatImage
import os
from datetime import datetime

# === 1. Toaletter och vatten från GeoJSON ===
data_url = "https://gist.githubusercontent.com/salgo60/e16b01890b2ac276342f473c23a7187a/raw/toilets_and_water_SAT_2025-07-24_0713.geojson"
geojson_data = requests.get(data_url).json()

# === 2. Skapa karta ===
m = folium.Map(location=[59.3, 18.5], zoom_start=9, control_scale=True)
Fullscreen().add_to(m)

# === 3. Popup-generator ===
def create_popup(properties, lat=None, lon=None):
    name = properties.get("name", "⚠️ Namn saknas")
    wikidata = properties.get("wikidata")
    osm_id = properties.get("osm_id")
    osm_type = properties.get("osm_type", "node")
    website = properties.get("website")
    image = properties.get("image")
    wheelchair = properties.get("wheelchair")

    html = f"<b>{name}</b><br>"

    if image:
        html += f'<img src="{image}" width="200"><br>'

    if wikidata:
        html += f'Wikidata: <a href="https://www.wikidata.org/wiki/{wikidata}" target="_blank">{wikidata}</a><br>'
    else:
        html += "⚠️ Saknar Wikidata<br>"

    if website:
        html += f'Website: <a href="{website}" target="_blank">{website}</a><br>'
    else:
        html += "⚠️ Saknar webbsida<br>"

    if osm_id:
        osm_url = f"https://www.openstreetmap.org/{osm_type}/{osm_id}"
        osm_edit = f"https://www.openstreetmap.org/edit?{osm_type}={osm_id}"
        html += f'OSM: <a href="{osm_url}" target="_blank">{osm_url}</a><br>'
        html += f'✏️ <a href="{osm_edit}" target="_blank">Redigera i OSM</a><br>'
    else:
        html += "⚠️ Saknar OSM-id<br>"

    # --- Förbättringsförslag ---
    html += "<hr><b>📋 Förbättringsförslag:</b><ul>"
    if not image:
        html += "<li>⚠️ Lägg till bild</li>"
    if not website:
        html += "<li>⚠️ Lägg till officiell webbsida</li>"
    if not wikidata:
        html += "<li>⚠️ Lägg till Wikidata-koppling</li>"
    if not osm_id:
        html += "<li>⚠️ Lägg till OSM-länk</li>"
    if not wheelchair:
        html += "<li>♿ Lägg till rullstolsåtkomst (taggen wheelchair)</li>"
    if lat and lon:
        mapillary_url = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=17"
        html += f'<li>📷 Kolla <a href="{mapillary_url}" target="_blank">Mapillary-bilder</a> för platsen</li>'
    html += "</ul>"

    # Wikimedia Commons för SAT-etapper
    etapp_namn = str(properties.get("name") or "")
    if etapp_namn.startswith("SAT_"):
        wiki_url = f"https://wikimap.toolforge.org/?cat={etapp_namn}&subcats=true&subcatdepth=3&cluster=false"  
        html += f'🖼️ <a href="{wiki_url}" target="_blank">Wikimedia Commons bilder</a><br>'

    return folium.Popup(IFrame(html, width=280, height=340))

# === 4. Lägg till markörer ===
for feature in geojson_data["features"]:
    props = feature["properties"]
    coords = feature["geometry"]["coordinates"]
    lon, lat = coords if feature["geometry"]["type"] == "Point" else coords[0][0]

    amenity = props.get("amenity", "")
    if amenity == "toilets":
        icon = folium.Icon(color="blue", icon="restroom", prefix="fa")
    elif amenity == "drinking_water":
        icon = folium.Icon(color="green", icon="tint", prefix="fa")
    else:
        icon = folium.Icon(color="gray", icon="question", prefix="fa")

    folium.Marker(
        location=[lat, lon],
        popup=create_popup(props, lat=lat, lon=lon),
        icon=icon
    ).add_to(m)

# === 5. Ladda och rita SAT-leden via Overpass ===
overpass_url = "https://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
relation(19012437);
way(r);
(._;>;);
out body;
"""
r = requests.get(overpass_url, params={'data': overpass_query})
data = r.json()

nodes = {el['id']: (el['lat'], el['lon']) for el in data['elements'] if el['type'] == 'node'}
ways = [el for el in data['elements'] if el['type'] == 'way']

for way in ways:
    coords = [(nodes[nid][0], nodes[nid][1]) for nid in way['nodes'] if nid in nodes]
    folium.PolyLine(locations=coords, color="red", weight=4, opacity=0.8, tooltip="SAT-leden").add_to(m)

# === 6. About-box med metadata ===
created_date = datetime.now().strftime("%Y-%m-%d %H:%M")
about_html = f"""
<div style="position: fixed; 
     bottom: 20px; left: 20px; width: 320px; 
     background-color: white; border:2px solid grey; z-index:9999; 
     padding: 10px; font-size:12px;">
<b>ℹ️ Om kartan</b><br>
Skapad: {created_date}<br>
GitHub issues:<br>
- <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/117" target="_blank">#117 Generate objects along the Trail - drinking water / toilets</a><br>
- <a href="https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/122" target="_blank">#122 Statisk genererad websida med problem</a>
</div>
"""
m.get_root().html.add_child(folium.Element(about_html))

# === 7. Spara HTML i underkatalog 'output' ===
timestamp = datetime.now().strftime("%Y%m%d_%H%M")

# Skapa underkatalog om den inte finns
output_dir = os.path.join(os.getcwd(), "output")
os.makedirs(output_dir, exist_ok=True)

output_path_ts = os.path.join(output_dir, f"toaletter_vatten_sat_karta_{timestamp}.html")
output_path_latest = os.path.join(output_dir, "toaletter_vatten_sat_karta_latest.html")

m.save(output_path_ts)
m.save(output_path_latest)

print(f"Kartor sparade i: {output_dir}")
print(f" - {output_path_ts}")
print(f" - {output_path_latest}")


Kartor sparade i: /Users/salgo/Documents/GitHub/Stockholm_Archipelago_Trail/notebook/output
 - /Users/salgo/Documents/GitHub/Stockholm_Archipelago_Trail/notebook/output/toaletter_vatten_sat_karta_20250916_0410.html
 - /Users/salgo/Documents/GitHub/Stockholm_Archipelago_Trail/notebook/output/toaletter_vatten_sat_karta_latest.html


In [7]:
# End timer and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Print current date and total time
print("Date:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print("Total time elapsed: {:.2f} seconds".format(elapsed_time))

Date: 2025-09-16 04:10:28
Total time elapsed: 21.52 seconds
